In [96]:
import synapseclient
from synapseclient import Project, File, Folder
from synapseclient import Schema, Column, Table, Row, RowSet, as_table_columns
import itertools
import pandas as pd
import numpy as np

In [32]:
syn = synapseclient.Synapse()
syn.login()

Welcome, Victor Baham!



In [33]:
divco_bio = pd.read_csv(syn.get('syn51757645').path)
rosmap_bio = pd.read_csv(syn.get('syn21323366').path)
mayo_bio = pd.read_csv(syn.get('syn20827192').path)
msbb_bio = pd.read_csv(syn.get('syn21893059').path)

print(len(divco_bio) + len(rosmap_bio) + len(mayo_bio) + len(msbb_bio))

26053


In [45]:
mega_bio = pd.concat([divco_bio, rosmap_bio, mayo_bio, msbb_bio], ignore_index=True)
mega_bio

[WARNING] /Users/vbaham/opt/anaconda3/lib/python3.9/site-packages/IPython/lib/pretty.py:700: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)

[WARNING] /Users/vbaham/opt/anaconda3/lib/python3.9/site-packages/IPython/core/formatters.py:345: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()



,individualID,specimenID,dataGenerationSite,organ,tissue,BrodmannArea,isPostMortem,sampleStatus,nucleicAcidSource,assay,...,tissueVolume,cellType,fastingState,samplingAge,samplingAgeUnits,visitNumber,exclude,excludeReason,samplingDate,Exclusion_Category
0,NYBB_17,NYBB_17WGS,NYGC,brain,dorsolateral prefrontal cortex,BA9,True,frozen,bulk cell,wholeGenomeSeq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NYBB_147,NYBB_147WGS,NYGC,brain,dorsolateral prefrontal cortex,BA9,True,frozen,bulk cell,wholeGenomeSeq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NYBB_148,NYBB_148WGS,NYGC,brain,dorsolateral prefrontal cortex,BA9,True,frozen,bulk cell,wholeGenomeSeq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NYBB_153,NYBB_153WGS,NYGC,brain,dorsolateral prefrontal cortex,BA9,True,frozen,bulk cell,wholeGenomeSeq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NYBB_76,NYBB_76WGS,NYGC,brain,dorsolateral prefrontal cortex,BA9,True,frozen,bulk cell,wholeGenomeSeq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sample swap,NaN,NaN


#### Find number of distinct specimens for each assay type:

In [98]:
unique_specs_per_assay = mega_bio.groupby(['assay', 'specimenID'])['specimenID'].nunique()
unique_specs_per_assay_piv_1 = mega_bio.groupby(['assay', 'specimenID'],as_index = False).nunique().pivot('assay'
                    ,'specimenID').fillna(0)
unique_specs_per_assay_piv_2 = mega_bio.groupby(['assay', 'specimenID']).size().unstack('specimenID', fill_value=0.0)
unique_specs_per_assay_piv_2

[WARNING] /var/folders/lg/w3yckgy936lc0r6yw790hjs00000gr/T/ipykernel_37968/783140119.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  unique_specs_per_assay_piv_1 = mega_bio.groupby(['assay', 'specimenID'],as_index = False).nunique().pivot('assay'



specimenID,01_120405,02_120405,03_120405,04_120405,05_120405,06_120405,07_120410,08_120410,09_120410,1000,...,sd_b25.129C,sd_b25.129N,sd_b25.130C,sd_b25.130N,sd_b25.131C,sd_b25.131N,sd_b25.132C,sd_b25.132N,sd_b25.133C,sd_b25.133N
assay,,,,,,,,,,,,,,,,,,,,,
10x multiome,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATACSeq,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATACseq,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Biocrates Bile Acids,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Biocrates p180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ChIPSeq,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Metabolon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TMT quantitation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
exomeSeq,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Find number of specimens of each tissue type for each assay type:

In [91]:
unique_specs_per_assay_and_tissue = mega_bio.groupby(['assay', 'specimenID', 'tissue'])['specimenID'].count()
unique_specs_per_assay_and_tissue_piv = mega_bio.groupby(['assay', 'tissue'])['specimenID'].count().unstack('tissue', fill_value=0.0)
unique_specs_per_assay_and_tissue_piv

tissue,Head of caudate nucleus,blood,caudate nucleus,cerebellar cortex,cerebellum,dorsolateral prefrontal cortex,frontal cortex,frontal pole,inferior frontal gyrus,occipital visual cortex,parahippocampal gyrus,posterior cingulate cortex,prefrontal cortex,serum,superior temporal gyrus,temporal cortex,temporal pole,unspecified
assay,,,,,,,,,,,,,,,,,,
10x multiome,0.0,0.0,128.0,0.0,0.0,135.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136.0,0.0,0.0,0.0
ATACSeq,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,288.0,0.0,0.0,0.0,348.0,0.0,0.0,0.0
ATACseq,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Biocrates Bile Acids,0.0,0.0,0.0,0.0,0.0,111.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Biocrates p180,0.0,0.0,0.0,0.0,0.0,111.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,597.0,0.0,0.0,0.0,0.0
ChIPSeq,0.0,0.0,0.0,0.0,0.0,712.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Metabolon,0.0,0.0,0.0,0.0,182.0,514.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.0,0.0,0.0
TMT quantitation,0.0,0.0,0.0,0.0,0.0,1505.0,0.0,0.0,0.0,0.0,190.0,0.0,0.0,0.0,280.0,0.0,0.0,0.0
exomeSeq,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83.0,0.0,0.0,1.0,0.0,0.0,0.0,262.0,0.0,0.0,0.0


In [85]:
len(mega_bio[(mega_bio['tissue'] == 'caudate nucleus') & (mega_bio['assay'] == '10x multiome')])

128